# On Call List Planner and Optimizer
An applet to organise and optimize your clinic's chaotic on-call list every month, using pandas DataFrames. 

In [ ]:
import pandas as pd
import numpy as np
import random
import os

# To see all of the dataframes when printed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Dictionary for specific values for specific days. 
For example, 2nd of December 2022 is friday, and the next day is free. Which means in state hospitals in Turkey, the person who is on call will be paid for their 15 hours of work. For normal weekdays it's 7

In [ ]:
# Dictionary for specific values for specific days. 
def day_value(argument):
    switcher = {
      2: 15,
      3: 24,
      4: 16,
      9: 15,
      10: 24,
      11: 16,
      16: 15,
      17: 24,
      18: 16,
      23: 15,
      24: 24,
      25: 16,
      30: 15,
      31: 24
    }
    return switcher.get(argument, 7)

### Importing the preliminary csv
csv should contain name and number of days as the first row.
Days with permission will be ignored if 0.1 written on that day's cell. 

In [ ]:
df_core = pd.read_csv('dec.csv', encoding='utf8', index_col=0, header=0).T
df_core.fillna(0, inplace = True)
print("CSV Import:\n",df_core.T,"\n")

### Main algorithm

In [ ]:
df_score = pd.DataFrame({"seed":[],"score":[],"range":[]})

def run(df_core, rand, verbatim = 0):


  df = df_core.sample(frac=1,axis=1,random_state=rand)
  
  for col in range(len(df.columns)):
    for row in range(len(df)):
      count_row = df.iloc[row,:][df.iloc[row,:] > 1].count()
      if df.iloc[row, col] == 0:
        count_col = df.iloc[:, col][df.iloc[:, col] > 1].count()
        try:
          if df.iloc[row-1, col] == 0 and df.iloc[row-2, col] == 0 and count_col < 8 and count_row <3:
            if row <29:
                if df.iloc[row+1, col] == 0 and df.iloc[row+2, col] == 0:
                  df.iloc[row, col] = day_value(row+1)
            else:
              df.iloc[row, col] = day_value(row+1)
        except IndexError:
          continue
      #print (count_row)
    sum_saat = df.iloc[:, col].sum()
  #  print (df.columns[col], count_col, sum_saat)
  
  for col in range(len(df.columns)):
    for row in range(len(df)):
      count_row = df.iloc[row,:][df.iloc[row,:] > 1].count()
      if df.iloc[row, col] == 0:
        count_col = df.iloc[:, col][df.iloc[:, col] > 1].count()
        try:
          if df.iloc[row-1, col] < 1 and df.iloc[row-2, col] < 1 and count_col < 8:
            if row <29:
                if df.iloc[row+1, col] < 1 and df.iloc[row+2, col] < 1:
                  df.iloc[row, col] = day_value(row+1)
            else:
              df.iloc[row, col] = day_value(row+1)
        except IndexError:
          continue
      #print (count_row)
    sum_saat = df.iloc[:, col].sum()
  #  print (df.columns[col], count_col, sum_saat)  
  
 
  #range_saat = np.ptp(df.sum())
  dfT = df.T
  range_saat = np.ptp(dfT[dfT.T.count()>7].T.sum())
  score =  df.to_numpy().sum()
  doksanbesalti = dfT[dfT.T.count()>7].T.sum()<95

  nob_say = dfT.iloc[:,][dfT.iloc[:,]>1].count()
  ikialtinobet = nob_say[nob_say<2].sum()

    
  df_exit = pd.DataFrame({"seed":[rand],"score":[score],"range":[range_saat], "undertwocalls":[ikialtinobet]})
    
  print ("Seed:",rand, "Score:", score, "Range", range_saat, "undertwocalls", ikialtinobet)
  
    
  if verbatim == 1:
    print("\nDays per person:\n", df.iloc[:,][df.iloc[:,]>1].count())
    print("\nPeople per day:\n", nob_say)
    print ("\nHour count:\n",df.sum())
    print("\nList:\n",dfT)
    print("--------------------------------")
    
  
  return df, df_exit
    

### Score and list

In [ ]:
pd.set_option('display.max_rows', None)
print("\nTotal hours of clinic:\n",df_score[df_score['undertwocalls']<1].sort_values(by='score', ascending=False))

### Select a small range to look closer

In [ ]:
range_of_seeds = {1,2,3,4} # MODIFY THIS

In [ ]:
for rand in range_of_seeds:
    df, df_exit = run(df_core,rand,1)
    df_score = pd.concat([df_score, df_exit], ignore_index = True)
    df_score.reset_index()

### Select a final dataframe

In [ ]:
final_seed = 1 # MODIFY THIS

In [ ]:
df, df_exit = run(df_core,final_seed,1)
df_score = pd.concat([df_score, df_exit], ignore_index = True)
df_score.reset_index()

### Output as CSV

In [ ]:
dfT=df.T
dfT.replace(0.1,np.nan,inplace=True)
dfT.replace(0,np.nan,inplace=True)
dfT.to_csv('out.csv', encoding='utf-8')